<font color="green">*To start working on this notebook, or any other notebook that we will use in the Moringa Data Science Course, we will need to save our own copy of it. We can do this by clicking File > Save a Copy in Drive. We will then be able to make edits to our own copy of this notebook.*</font>

# Python Programming: Elastic Net Regression 

## Example

In [1]:
# Example 1
# ---
# Use the fair dataset from the pydataset library to predict marriage satisfaction based on the given variables.
# ---
# 
!pip install pydataset

     |████████████████████████████████| 15.9 MB 1.4 MB/s 
  Created wheel for pydataset: filename=pydataset-0.2.0-py3-none-any.whl size=15939431 sha256=543152772d33db150e847c4a3a854449ac5b321f874d1af656088f4df65b1fe6
  Stored in directory: /root/.cache/pip/wheels/32/26/30/d71562a19eed948eaada9a61b4d722fa358657a3bfb5d151e2
Successfully built pydataset


In [2]:
# Importing our libraries
# 
from pydataset import data
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 5000)
pd.set_option('display.max_columns', 5000)
pd.set_option('display.width', 10000)

from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

initiated datasets repo at: /root/.pydataset/


In [3]:
# Data preparation
# 
df=pd.DataFrame(data('Fair'))
df.loc[df.sex== 'male', 'sex'] = 0
df.loc[df.sex== 'female','sex'] = 1
df['sex'] = df['sex'].astype(int)
df.loc[df.child== 'no', 'child'] = 0
df.loc[df.child== 'yes','child'] = 1
df['child'] = df['child'].astype(int)
X=df[['religious','age','sex','ym','education','occupation','nbaffairs']]
y=df['rate']

In [4]:
# Creating our linear regression model for the purpose of comparison
# 
regression=LinearRegression()
regression.fit(X,y)
first_model=(mean_squared_error(y_true=y,y_pred=regression.predict(X)))
print(first_model) 

# This mean standard error score of 1.05 is our benchmark for determining 
# if the elastic net model will be better or worst. 

1.049873864469667


In [5]:
# Below are the coefficients of this first model. We use a for loop to go through 
# the model and the zip function to combine the two columns.
# 
coef_dict_baseline = {}
for coef, feat in zip(regression.coef_,X.columns):
    coef_dict_baseline[feat] = coef
coef_dict_baseline

{'age': -0.00905964542867384,
 'education': 0.06810255742293703,
 'nbaffairs': -0.07882571247653965,
 'occupation': -0.00597950685299818,
 'religious': 0.04235281110639179,
 'sex': 0.0888201333708709,
 'ym': -0.030458802565476555}

In [7]:
# Elastic Net Model
# Elastic net, just like ridge and lasso regression, requires normalize data. 
# This argument  is set inside the ElasticNet function. 
# The second thing we need to do is create our grid.
# 
elastic=ElasticNet(normalize=True)
search=GridSearchCV(estimator=elastic,param_grid={'alpha':np.logspace(-5,2,8),'l1_ratio':[.2,.4,.6,.8]},scoring='neg_mean_squared_error',n_jobs=1,refit=True,cv=10)

In [10]:
# We will now fit our model and display the best parameters and the best results we can get with that setup.
# 
search.fit(X,y)
search.best_params_
abs(search.best_score_)

1.0819158709244472

In [12]:
# The best hyperparameters was an alpha set to 0.001 and a l1_ratio of 0.8. 
# With these settings we got an MSE of 1.08. This is above our baseline model of MSE 1.05  for the baseline model. 
# Which means that elastic net is doing worse than linear regression. 
# For clarity, we will set our hyperparameters to the recommended values and run on the data.
# 
elastic=ElasticNet(normalize=True,alpha=0.001,l1_ratio=0.75)
elastic.fit(X,y)
second_model=(mean_squared_error(y_true=y,y_pred=elastic.predict(X)))
print(second_model)

1.0566430678343806


In [13]:
# Below are the coefficients
# 
coef_dict_baseline = {}
for coef, feat in zip(elastic.coef_,X.columns):
    coef_dict_baseline[feat] = coef
coef_dict_baseline

# The coefficients are mostly the same. 
# Notice that occupation was completely removed from the model in the elastic net version. 
# This means that this values was no good to the algorithm. Traditional regression cannot do this.

{'age': -0.008630896492807693,
 'education': 0.04429085595448633,
 'nbaffairs': -0.06679513627963515,
 'occupation': -0.0,
 'religious': 0.019475417249578564,
 'sex': 0.01811646456809079,
 'ym': -0.02422483127451297}

## Challenges

### <font color="green">Challenge 1</font>

In [23]:
# Challenge 1
# ---
# Question: Using the given housiet, create a regression model to predict 
# the value of prices of a house using the given features. 
# ---
# Dataset 
url = 'http://bit.ly/BostonHousingDataset'
# ---
# 
bdf = pd.read_csv(url)
bdf.head()
# define X and Y \
X = bdf.drop ('tax', 1)
y = bdf['tax']

lr = LinearRegression()
lr.fit(X, y)
first_model=(mean_squared_error(y_true=y,y_pred=lr.predict(X)))
print(first_model) 



3079.517025404413


In [24]:
# Below are the coefficients of this first model. We use a for loop to go through 
# the model and the zip function to combine the two columns.
# 
coef_dict_baseline = {}
for coef, feat in zip(lr.coef_,X.columns):
    coef_dict_baseline[feat] = coef
coef_dict_baseline

{'age': 0.10415212922794237,
 'b': -0.002470321410012577,
 'chas': -22.51192296750458,
 'crim': -0.28873412133877585,
 'dis': -1.5494749241255885,
 'indus': 7.044131663777457,
 'lstat': -1.1126056086738825,
 'medv': -1.7348657156688934,
 'nox': 43.32664401203344,
 'ptratio': 0.9161807522921855,
 'rad': 14.135332861439798,
 'rm': -1.4689828318668419,
 'zn': 0.8778597945656135}

In [25]:
# initialize elastic net and grid sEArch cv
elastic=ElasticNet(normalize=True)
search=GridSearchCV(estimator=elastic,param_grid={'alpha':np.logspace(-5,2,8),'l1_ratio':[.2,.4,.6,.8]},scoring='neg_mean_squared_error',n_jobs=1,refit=True,cv=10)

In [29]:
# We will now fit our model and display the best parameters and the best results we can get with that setup.
# 
search.fit(X,y)
search.best_params_
abs(search.best_score_)


4533.698051735112

In [30]:
# The best hyperparameters was an alpha set to'alpha': 1e-05, 'l1_ratio': 0.8 
# With these settings we got an MSE of 4533.70 This is above our baseline model of MSE 3079  for the baseline model. 
# Which means that elastic net is doing worse than linear regression. 
# For clarity, we will set our hyperparameters to the recommended values and run on the data.
# 
elastic=ElasticNet(normalize=True,alpha=1e-05,l1_ratio=0.8)
elastic.fit(X,y)
second_model=(mean_squared_error(y_true=y,y_pred=elastic.predict(X)))
print(second_model)

3079.566872496664


In [31]:
# Below are the coefficients
# 
coef_dict_baseline = {}
for coef, feat in zip(elastic.coef_,X.columns):
    coef_dict_baseline[feat] = coef
coef_dict_baseline

# The coefficients are mostly the same. 
# Notice that occupation was completely removed from the model in the elastic net version. 
# This means that this values was no good to the algorithm. Traditional regression cannot do this.


{'age': 0.10256198524728598,
 'b': -0.0030256773939518963,
 'chas': -22.488586803632472,
 'crim': -0.27357648121241107,
 'dis': -1.533331707230361,
 'indus': 7.029869518304757,
 'lstat': -1.097357566228503,
 'medv': -1.721133985535651,
 'nox': 45.158791465804136,
 'ptratio': 0.9820422251861676,
 'rad': 14.096134529791406,
 'rm': -1.4524484629677312,
 'zn': 0.876530005651836}

### <font color="green">Challenge 2</font>

In [ ]:
# Challenge 2
# ---
# Question: Using the Ames Housing dataset, create a regression model to predict the sales price of home 
# applying elastic net regression.
# ---
# Dataset Source = http://bit.ly/HousePricesDataset
# 
OUR CODE GOES HERE

### <font color="green">Challenge 3</font>

In [ ]:
# Challenge 3
# ---
# Question: Given the medical cost personal dataset, accurately predict insurance cost using a regression model.
# ---
# Dataset Source = http://bit.ly/https://bit.ly/insurance-_dataset
# 
OUR CODE GOES HERE

### <font color="green">Challenge 4</font>

In [ ]:
# Challenge 4
# ---
# Question: Use ElasticNet regression to build a model that is able to accurately predict the profits of a startup.
# ---
# Dataset Source = http://bit.ly/StartupsDataset
# ---
# 
OUR CODE STARTS HERE

### <font color="green">Challenge 5</font>

In [ ]:
# Challenge 5
# ---
# Question: Build a prediction model to predict duration for any combination of country,operator, 
# services and category given the genre,language and number of units. 
# Apply ElasticNet regression while building your model. 
# ---
# Dataset Source = https://bit.ly/Audio_content_consumption
# ---
# 
OUR CODE STARTS HERE